<a href="https://colab.research.google.com/github/hussain0048/Projects-/blob/master/Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face detection with OpenCV and deep learning
# **Introduction:**

From there I’ll demonstrate how you can perform face detection in images using OpenCV and deep learning.In this first example we’ll learn how to apply face detection with OpenCV to single input images.

In [ ]:
!git clone https://github.com/hussain0048/Projects-

# 1 **Terminologies**#

First introducing you with the terminologies used in this advanced python project of gender and age detection 

## 1.1 What is Computer Vision? 
Computer Vision is the field of study that enables computers to see and identify digital images and videos as a human would. The challenges it faces largely follow from the limited understanding of biological vision. Computer Vision involves acquiring, processing, analyzing, and understanding digital images to extract high-dimensional data from the real world in order to generate symbolic or numerical information which can then be used to make decisions. The process often includes practices like object recognition, video tracking, motion estimation, and image restoration.

##1.2 What is OpenC(VOpen Source Computer Vision)?
OpenCV is short for Open Source Computer Vision. Intuitively by the name, it is an open-source Computer Vision and Machine Learning library. This library is capable of processing real-time image and video while also boasting analytical capabilities. It supports the Deep Learning frameworks TensorFlow, Caffe, and PyTorch.
You can perform fast, accurate face detection with OpenCV using a pre-trained deep learning face detector model shipped with the library.

You may already know that OpenCV ships out-of-the-box with pre-trained Haar cascades that can be used for face detection…

With OpenCV 3.3, we can utilize pre-trained networks with popular deep learning frameworks. The fact that they are pre-trained implies that we don’t need to spend many hours training the network — rather we can complete a forward pass and utilize the output to make a decision within our application [7]

### 1.2.1 dnn Module

This module now supports a number of deep learning frameworks, including Caffe, TensorFlow, and Torch/PyTorch.

### 1.2.1  **Caffe models**
When using OpenCV’s deep neural network module with Caffe models, you’ll need two sets of files:

The .prototxt file(s) which define the model architecture (i.e., the layers themselves)
The .caffemodel file which contains the weights for the actual layers
Both files are required when using models trained using Caffe for deep learning.


### 1.2.2 How does the OpenCV deep learning face detector work [6]?

OpenCV’s deep learning face detector is based on the Single Shot Detector (SSD) framework with a ResNet base network (unlike other OpenCV SSDs that you may have seen which typically use MobileNet as the base network).


## 1.3 What is a CNN?
A Convolutional Neural Network is a deep neural network (DNN) widely used for the purposes of image recognition and processing and NLP. Also known as a ConvNet, a CNN has input and output layers, and multiple hidden layers, many of which are convolutional. In a way, CNNs are regularized multilayer perceptrons.


## 1.4 The CNN Architecture
The convolutional neural network for this python project has 3 convolutional layers:
- Convolutional layer; 96 nodes, kernel size 7
- Convolutional layer; 256 nodes, kernel size 5
- Convolutional layer; 384 nodes, kernel size 3

It has 2 fully connected layers, each with 512 nodes, and a final output layer of softmax type

# 2 - **Import library** 


In [1]:
import imutils
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

#3 - **Start webcam and Caputure the picture** # 

In [2]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

Click 'Capture' to make photo using your webcam.

In [ ]:
image_file = take_photo()

Read, resize and display the image.

In [ ]:
#image = cv2.imread(image_file, cv2.IMREAD_UNCHANGED)
image = cv2.imread(image_file)
# resize it to have a maximum width of 400 pixels
image = imutils.resize(image, width=400)
(h, w) = image.shape[:2]
print(w,h)
cv2_imshow(image)

##4-Download the pre-trained face detection model
OpenCV’s deep learning face detector is based on the Single Shot Detector (SSD) framework with a ResNet base network. The network is defined and trained using the Caffe Deep Learning framework
Download the pre-trained face detection model, consisting of two files:


The network definition (deploy.prototxt)

The learned weights (res10_300x300_ssd_iter_140000.caffemodel)

In [ ]:
!wget -N https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
!wget -N https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

# 5- **Load the pre-trained face detection network model from disk**

In [ ]:
 print("[INFO] loading model...")
prototxt = 'deploy.prototxt'
model = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model)

#6- **Resizing the image to a fixed 300x300 pixels** 

Use the dnn.blobFromImage function to construct an input blob by resizing the image to a fixed 300x300 pixels and then normalizing it.

In [7]:
# resize it to have a maximum width of 400 pixels
image = imutils.resize(image, width=400)
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

# 7- **Computing object detections**
Pass the blob through the neural network and obtain the detections and predictions.

In [ ]:
print("[INFO] computing object detections...")
net.setInput(blob)
detections = net.forward()

# 8 - **Loop over the detections and draw boxes around the detected faces**.

In [9]:
for i in range(0, detections.shape[2]):

	# extract the confidence (i.e., probability) associated with the prediction
	confidence = detections[0, 0, i, 2]

	# filter out weak detections by ensuring the `confidence` is
	# greater than the minimum confidence threshold
	if confidence > 0.5:
		# compute the (x, y)-coordinates of the bounding box for the object
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		# draw the bounding box of the face along with the associated probability
		text = "{:.2f}%".format(confidence * 100)
		y = startY - 10 if startY - 10 > 10 else startY + 10
		cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
		cv2.putText(image, text, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

Show the resulting image

In [ ]:
cv2_imshow(image)

References:

 - [1] Gender and Age Detection Python Project

https://data-flair.training/blogs/python-project-gender-age-detection/?fbclid=IwAR2YqS3yzlvQwqgkWuSjv4azyxkzSMamn97StQ0Vqm5KGy_7-2Sj4EoHVrI
 - [2] Adience Benchmark Gender And Age Classification
https://towardsdatascience.com/predict-age-and-gender-using-convolutional-neural-network-and-opencv-fd90390e3ce6
- [3] Gender and agge detection 
https://github.com/GilLevi/AgeGenderDeepLearning/tree/master/models

- [4] Predict Age and Gender Using Convolutional Neural Network and OpenCV
https://www.kdnuggets.com/2019/04/predict-age-gender-using-convolutional-neural-network-opencv.html

- [5] Age and Gender Classification Using Convolutional Neural Networks
https://talhassner.github.io/home/publication/2015_CVPR

-[6]Face detection with OpenCV and deep learning
]https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/

[7] Deep Learning with OpenCV
https://www.pyimagesearch.com/2017/08/21/deep-learning-with-opencv/

[8] Object detection with deep learning and OpenCV
https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/

